In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from natsort import natsorted
import ast

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'Size Length Rugs_Runners'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

strategy_version_input=767
attribute_id_input=4627

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [5]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]   

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    tuples=tuple(split_digits(s, *args, **kwargs))
    print(tuples)
    return tuples 

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

from fractions import Fraction
def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))
# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]



In [6]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df = df[df['bucket_id'].isin(buckets)]
len(df)

3797

In [7]:
regex_pattern =r'''(?i)(\d+(?:(?<!')'(?!'))?\s?\d*\s?\\{0,2}\"?\s?x\s?\d+(?:(?<!')'(?!'))?\s?\d*\s?\\{0,2}\"?)|()'''
df['inch'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df_oz=df[df['inch'].astype(str)!='[]']
df_na=df[df['inch'].astype(str)=='[]']

print(len(df_oz))
print(len(df_na))

dfna=df[df['inch'].astype(str)=='[]']
dfna[curation_col]='n/a'
match_na=dfna[['external_id',curation_col]]

3392
405


In [8]:
input_ = ["10'x10'", "60\" x 72\"", "5'x8'","5x8","60\"x92\"", "8'10\"x12'", "8'1x10'1\""]

def ft_inch(input_):
    inches_only = []
#     print(input_)
    for s in input_:
        s.replace(" ", "")
        sides = s.split("x")
        new_sides = []
        for side in sides:
            inches = 0
            split1 = side.split("'")

            
            if len(split1) > 1 or (len(split1) == 1 and not side.__contains__('"')):
                inches = int(split1[0].replace('[','').replace('','0')) * 12
#             print('C')
            split2 = side.split('"')
            if len(split2) > 1:
#                 print(split2)
                inches += int(split2[0].split("'")[-1])
            elif len(split2) == 1 and len(split1) > 1 and len(split1[1]) > 0:
                inches += int(split1[1])
            new_sides.append(str(inches) + '"')
        inches_only.append("x".join(new_sides))
#         print(inches_only)
    print(inches_only)

In [9]:
input_ = ["10'x10'", "60\" x 72\"", "5'x8'","5x8","60\"x92\"", "8'10\"x12'", "8'1x10'1\""]
y=["5\'3\"x7\'7\""]
ft_inch(input_)

['12000"x12000"', '60"x72"', '600"x960"', '600"x960"', '60"x92"', '970"x12240"', '961"x12001"']


In [122]:
df_oz['inches']=df_oz['inch'].apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"'\,",'"",',str(x))).apply(lambda x: re.sub(r"\,'",',"',str(x))).apply(lambda x: re.sub(r'"""','""',str(x)))
df_oz['inches']=df_oz['inches'].apply(lambda x: re.sub(r'\\{2}',r"\\", str(x)))                          
df_oz['inches'].apply(lambda x: ft_inch(x))

["5\"x94"", "5'x8' "]
C
['0"']
C
['', '']


ValueError: invalid literal for int() with base 10: ''

In [116]:
error

NameError: name 'error' is not defined

# send to the folder for upload

In [ ]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name

# def looks_good(customer, attribute, df, matches): 
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
# #     non_matches = df[df['matches'].astype(str) == '[]']
# #     non_matches[curation_col] = r'n/a'
# #     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
#     matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

# looks_good('Bed Bath & Beyond', attribute, df, matchesnewest)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('Bed Bath & Beyond', attribute, df, match_na)